In [2]:
import pandas as pd
import numpy as np
import h5py

from keras.models import load_model
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding



Using TensorFlow backend.


In [3]:
def read_weight_file(file_path):
    f = h5py.File(file_path,'r')
    return f

In [32]:
f = read_weight_file('../../../LSTMVis/data/05childbook/05childbook/states.h5')
f.keys()

<KeysViewHDF5 ['output1', 'states1']>

In [34]:
f['states1']

<HDF5 dataset "states1": shape (1271900, 200), type "<f4">

In [15]:
f['output1']

<HDF5 dataset "output1": shape (1271900, 200), type "<f4">

In [20]:
df_states = pd.DataFrame(np.array(f['states1']))
df_states.head()

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.999395,0.139939,-0.133315,-0.707135,0.092862,-0.653351,-0.812206,-0.873559,0.677905,0.112582,...,0.168090,0.058637,0.138943,-0.447951,0.771288,-0.997505,-0.792947,0.015164,-0.005032,-0.679110
1,0.079009,0.534143,0.827991,-0.070337,0.105284,0.412934,-1.022239,-0.830880,-0.026431,-0.006075,...,-0.019834,0.996436,0.041451,0.081331,0.156615,0.748737,-0.289053,-0.032697,0.000016,-0.674544
2,-0.913969,0.486170,0.108466,-0.306623,0.690331,0.952605,-0.343406,0.370140,-0.376919,0.525713,...,0.066910,0.591809,0.896092,0.371651,-0.720150,0.428870,-0.842386,-0.042881,-0.099725,-0.727310
3,-0.456632,-0.896531,0.167117,0.408090,0.914130,0.024740,0.794913,-0.578901,-0.324473,0.224386,...,0.273680,0.825968,-0.203984,0.026162,0.940264,0.559527,-0.701537,-0.051908,0.020711,-0.741639
4,-0.598989,-1.001568,0.034984,-0.366462,0.935715,0.999910,1.028395,-1.051747,-0.697554,1.023287,...,0.390594,0.728473,-0.713890,0.082018,0.979188,0.212284,-1.270396,-0.065537,-0.003914,-0.826838


In [22]:
df_states.shape

(1271900, 200)

In [21]:
df_output = pd.DataFrame(np.array(f['output1']))
df_output.head()

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.581290,0.000340,-0.027683,-0.430689,0.033046,-0.043899,-0.112473,-0.680815,0.539585,0.006208,...,0.095224,0.056019,0.137070,-0.226631,0.643734,-0.495256,-0.507912,0.009428,-0.001253,-0.321540
1,0.032917,0.008198,0.093196,-0.047473,0.103909,0.000585,-0.094483,-0.680342,-0.026275,-0.001114,...,-0.016411,0.312569,0.041424,0.013392,0.123176,0.043307,-0.259534,-0.032407,0.000011,-0.485580
2,-0.501529,0.075285,0.036841,-0.226716,0.015573,0.318165,-0.023613,0.349541,-0.229377,0.423139,...,0.036713,0.434348,0.628503,0.111479,-0.589384,0.358589,-0.510057,-0.031762,-0.000549,-0.502966
3,-0.401368,-0.456501,0.006114,0.149163,0.625086,0.021770,0.007484,-0.042390,-0.034800,0.162708,...,0.156461,0.501618,-0.016214,0.018712,0.420728,0.021748,-0.513047,-0.042374,0.019856,-0.559717
4,-0.251165,-0.067411,0.033815,-0.004376,0.555721,0.070168,0.004318,-0.772693,-0.516666,0.757734,...,0.169819,0.068413,-0.152470,0.065834,0.752220,0.209106,-0.767739,-0.053505,-0.002154,-0.658323


In [7]:
s = read_weight_file('../../../LSTMVis/data/05childbook/05childbook/train.h5')
s.keys()

<KeysViewHDF5 ['indices', 'set_size', 'target', 'target_output', 'target_size', 'words']>

In [18]:
ind = s['indices'].shape
ind_df = pd.DataFrame(np.array(ind))
ind_df

,0
0,1817
1,20
2,35


In [29]:
pd.DataFrame(np.array(s['words']))

,0
0,3
1,4
2,5
3,1
4,6
...,...
1271907,355
1271908,632
1271909,2680
1271910,22


In [2]:
# generate a sequence from the model
def generate_seq(model, tokenizer, seed_text, n_words):
    in_text, result = seed_text, seed_text
    # generate a fixed number of words
    for _ in range(n_words):
        # encode the text as integer
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        encoded = array(encoded)
        # predict a word in the vocabulary
        yhat = model.predict_classes(encoded, verbose=0)
        # map predicted word index to word
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break
        # append to input
        in_text, result = out_word, result + ' ' + out_word
    return result

# source text
data = """ Jack and Jill went up the hill\n
        To fetch a pail of water\n
        Jack fell down and broke his crown\n
        And Jill came tumbling after\n """

# integer encode text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
encoded = tokenizer.texts_to_sequences([data])[0]


# determine the vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)

# create word -> word sequences
sequences = list()
for i in range(1, len(encoded)):
	sequence = encoded[i-1:i+1]
	sequences.append(sequence)
print('Total Sequences: %d' % len(sequences))

# split into X and y elements
sequences = array(sequences)
X, y = sequences[:,0],sequences[:,1]

# one hot encode outputs
y = to_categorical(y, num_classes=vocab_size)

# define model
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=1))
model.add(LSTM(50))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# fit network
model.fit(X, y, epochs=500, verbose=2)

# evaluate
print(generate_seq(model, tokenizer, 'Jack', 6))

Vocabulary Size: 22
Total Sequences: 24
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1, 10)             220       
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                12200     
_________________________________________________________________
dense_1 (Dense)              (None, 22)                1122      
Total params: 13,542
Trainable params: 13,542
Non-trainable params: 0
_________________________________________________________________
None


/Users/jaywang/anaconda3/envs/data_science/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/500
 - 1s - loss: 3.0916 - accuracy: 0.0000e+00
Epoch 2/500
 - 0s - loss: 3.0909 - accuracy: 0.0417
Epoch 3/500
 - 0s - loss: 3.0901 - accuracy: 0.1250
Epoch 4/500
 - 0s - loss: 3.0894 - accuracy: 0.1250
Epoch 5/500
 - 0s - loss: 3.0887 - accuracy: 0.1250
Epoch 6/500
 - 0s - loss: 3.0880 - accuracy: 0.1250
Epoch 7/500
 - 0s - loss: 3.0872 - accuracy: 0.1250
Epoch 8/500
 - 0s - loss: 3.0865 - accuracy: 0.2083
Epoch 9/500
 - 0s - loss: 3.0857 - accuracy: 0.2083
Epoch 10/500
 - 0s - loss: 3.0849 - accuracy: 0.2083
Epoch 11/500
 - 0s - loss: 3.0841 - accuracy: 0.2083
Epoch 12/500
 - 0s - loss: 3.0833 - accuracy: 0.2083
Epoch 13/500
 - 0s - loss: 3.0825 - accuracy: 0.2083
Epoch 14/500
 - 0s - loss: 3.0817 - accuracy: 0.2083
Epoch 15/500
 - 0s - loss: 3.0809 - accuracy: 0.2083
Epoch 16/500
 - 0s - loss: 3.0800 - accuracy: 0.2083
Epoch 17/500
 - 0s - loss: 3.0792 - accuracy: 0.2083
Epoch 18/500
 - 0s - loss: 3.0783 - accuracy: 0.2083
Epoch 19/500
 - 0s - loss: 3.0774 - accuracy: 0.208

Epoch 155/500
 - 0s - loss: 2.1129 - accuracy: 0.3333
Epoch 156/500
 - 0s - loss: 2.0997 - accuracy: 0.3333
Epoch 157/500
 - 0s - loss: 2.0866 - accuracy: 0.3333
Epoch 158/500
 - 0s - loss: 2.0735 - accuracy: 0.3333
Epoch 159/500
 - 0s - loss: 2.0603 - accuracy: 0.3333
Epoch 160/500
 - 0s - loss: 2.0472 - accuracy: 0.3750
Epoch 161/500
 - 0s - loss: 2.0341 - accuracy: 0.3750
Epoch 162/500
 - 0s - loss: 2.0210 - accuracy: 0.3750
Epoch 163/500
 - 0s - loss: 2.0079 - accuracy: 0.3750
Epoch 164/500
 - 0s - loss: 1.9947 - accuracy: 0.3750
Epoch 165/500
 - 0s - loss: 1.9816 - accuracy: 0.4583
Epoch 166/500
 - 0s - loss: 1.9685 - accuracy: 0.4583
Epoch 167/500
 - 0s - loss: 1.9553 - accuracy: 0.4583
Epoch 168/500
 - 0s - loss: 1.9422 - accuracy: 0.4583
Epoch 169/500
 - 0s - loss: 1.9290 - accuracy: 0.5000
Epoch 170/500
 - 0s - loss: 1.9159 - accuracy: 0.5000
Epoch 171/500
 - 0s - loss: 1.9027 - accuracy: 0.5000
Epoch 172/500
 - 0s - loss: 1.8895 - accuracy: 0.5000
Epoch 173/500
 - 0s - loss: 

Epoch 307/500
 - 0s - loss: 0.5891 - accuracy: 0.8333
Epoch 308/500
 - 0s - loss: 0.5847 - accuracy: 0.8333
Epoch 309/500
 - 0s - loss: 0.5804 - accuracy: 0.8333
Epoch 310/500
 - 0s - loss: 0.5761 - accuracy: 0.8333
Epoch 311/500
 - 0s - loss: 0.5719 - accuracy: 0.8333
Epoch 312/500
 - 0s - loss: 0.5677 - accuracy: 0.8333
Epoch 313/500
 - 0s - loss: 0.5635 - accuracy: 0.8333
Epoch 314/500
 - 0s - loss: 0.5594 - accuracy: 0.8333
Epoch 315/500
 - 0s - loss: 0.5554 - accuracy: 0.8333
Epoch 316/500
 - 0s - loss: 0.5514 - accuracy: 0.8333
Epoch 317/500
 - 0s - loss: 0.5475 - accuracy: 0.8333
Epoch 318/500
 - 0s - loss: 0.5436 - accuracy: 0.8333
Epoch 319/500
 - 0s - loss: 0.5397 - accuracy: 0.8333
Epoch 320/500
 - 0s - loss: 0.5359 - accuracy: 0.8333
Epoch 321/500
 - 0s - loss: 0.5321 - accuracy: 0.8333
Epoch 322/500
 - 0s - loss: 0.5284 - accuracy: 0.8333
Epoch 323/500
 - 0s - loss: 0.5247 - accuracy: 0.8333
Epoch 324/500
 - 0s - loss: 0.5210 - accuracy: 0.8333
Epoch 325/500
 - 0s - loss: 

Epoch 459/500
 - 0s - loss: 0.2840 - accuracy: 0.8750
Epoch 460/500
 - 0s - loss: 0.2833 - accuracy: 0.8750
Epoch 461/500
 - 0s - loss: 0.2827 - accuracy: 0.8750
Epoch 462/500
 - 0s - loss: 0.2820 - accuracy: 0.8750
Epoch 463/500
 - 0s - loss: 0.2813 - accuracy: 0.8750
Epoch 464/500
 - 0s - loss: 0.2806 - accuracy: 0.8750
Epoch 465/500
 - 0s - loss: 0.2800 - accuracy: 0.8750
Epoch 466/500
 - 0s - loss: 0.2793 - accuracy: 0.8750
Epoch 467/500
 - 0s - loss: 0.2787 - accuracy: 0.8750
Epoch 468/500
 - 0s - loss: 0.2780 - accuracy: 0.8750
Epoch 469/500
 - 0s - loss: 0.2774 - accuracy: 0.8750
Epoch 470/500
 - 0s - loss: 0.2768 - accuracy: 0.8750
Epoch 471/500
 - 0s - loss: 0.2761 - accuracy: 0.8750
Epoch 472/500
 - 0s - loss: 0.2755 - accuracy: 0.8750
Epoch 473/500
 - 0s - loss: 0.2749 - accuracy: 0.8750
Epoch 474/500
 - 0s - loss: 0.2743 - accuracy: 0.8750
Epoch 475/500
 - 0s - loss: 0.2737 - accuracy: 0.8750
Epoch 476/500
 - 0s - loss: 0.2731 - accuracy: 0.8750
Epoch 477/500
 - 0s - loss: 